In [4]:
import networkx as nx
import mysql.connector
import configparser

In [5]:
config = configparser.ConfigParser()

In [6]:
config.read("config.ini")
db_config = {
    "host": config["mysql"]["host"],
    "user": config["mysql"]["user"],
    "password": config["mysql"]["password"],
    "database": config["mysql"]["database"]
}
mydb = mysql.connector.connect(**db_config)
cursor = mydb.cursor(dictionary=True,buffered=True)

In [7]:
if 'topology_logical' not in globals():
    query_log_topo  = """
        SELECT 
           aendname, 
           bendname, 
           aendip, 
           bendip, 
           block_name, 
           physicalringname, 
           lrname 
        FROM topology_data_logical
    """
    cursor.execute(query_log_topo)
    topology_logical = cursor.fetchall()
    mydb.commit()
    topology_logical_graph = nx.Graph()
    for row in topology_logical:
        aend = row["aendip"]
        bend = row["bendip"]
        aendname = row["aendname"].upper()
        bendname = row["bendname"].upper()
        pr_name = row["physicalringname"].upper()
        lr_name = row["lrname"].upper()
        block_name = row["block_name"].upper()
        topology_logical_graph.add_node(aend, name=aendname,lr_name=lr_name,pr_name = pr_name,block_name=block_name)
        topology_logical_graph.add_node(bend, name=bendname,lr_name=lr_name,pr_name = pr_name,block_name=block_name)
        topology_logical_graph.add_edge(aend, bend,lr_name=lr_name,pr_name = pr_name,block_name=block_name)

In [8]:
block_router_query = """
    SELECT ip 
    FROM network_element 
    WHERE devicetype = 'BLOCK_ROUTER'
"""
cursor.execute(block_router_query)
block_routers = cursor.fetchall()
mydb.commit()

In [10]:
block_router_ips = {router['ip'] for router in block_routers}
print(len(block_router_ips))

20


In [11]:
block_router_ips

{'10.128.0.10',
 '10.128.0.11',
 '10.128.0.12',
 '10.128.0.13',
 '10.128.0.14',
 '10.128.0.16',
 '10.128.0.19',
 '10.128.0.26',
 '10.128.0.27',
 '10.128.0.48',
 '10.128.0.49',
 '10.128.0.50',
 '10.128.0.61',
 '10.128.0.75',
 '10.128.0.8',
 '10.128.0.83',
 '10.128.0.84',
 '10.128.0.85',
 '10.128.0.86',
 '10.128.0.87'}

In [12]:
reachable = nx.node_connected_component(topology_logical_graph,'10.128.0.10')

In [ ]:
len(reachabk)

{'10.128.0.10',
 '10.128.10.1',
 '10.128.10.10',
 '10.128.10.11',
 '10.128.10.12',
 '10.128.10.13',
 '10.128.10.15',
 '10.128.10.16',
 '10.128.10.18',
 '10.128.10.19',
 '10.128.10.2',
 '10.128.10.20',
 '10.128.10.21',
 '10.128.10.22',
 '10.128.10.23',
 '10.128.10.24',
 '10.128.10.25',
 '10.128.10.26',
 '10.128.10.27',
 '10.128.10.28',
 '10.128.10.29',
 '10.128.10.30',
 '10.128.10.31',
 '10.128.10.32',
 '10.128.10.33',
 '10.128.10.34',
 '10.128.10.35',
 '10.128.10.36',
 '10.128.10.37',
 '10.128.10.38',
 '10.128.10.39',
 '10.128.10.4',
 '10.128.10.40',
 '10.128.10.41',
 '10.128.10.43',
 '10.128.10.44',
 '10.128.10.45',
 '10.128.10.46',
 '10.128.10.47',
 '10.128.10.48',
 '10.128.10.49',
 '10.128.10.5',
 '10.128.10.50',
 '10.128.10.51',
 '10.128.10.52',
 '10.128.10.54',
 '10.128.10.55',
 '10.128.10.56',
 '10.128.10.57',
 '10.128.10.58',
 '10.128.10.59',
 '10.128.10.6',
 '10.128.10.60',
 '10.128.10.61',
 '10.128.10.62',
 '10.128.10.63',
 '10.128.10.64',
 '10.128.10.65',
 '10.128.10.66',
 '1

In [14]:
def can_reach_between_rings(graph, ring_type='lr_name'):
    """
    Analyze if nodes from one ring can reach nodes in different rings.
    
    Parameters:
    -----------
    graph : NetworkX Graph
        The network topology graph
    ring_type : str
        Either 'lr_name' for logical rings or 'pr_name' for physical rings
        
    Returns:
    --------
    dict
        Dictionary mapping each ring to the set of rings it can reach
    """
    # Group nodes by ring
    rings = {}
    for node, attrs in graph.nodes(data=True):
        ring = attrs.get(ring_type)
        if ring:
            if ring not in rings:
                rings[ring] = []
            rings[ring].append(node)
    
    # Find paths between rings
    ring_connectivity = {}
    for source_ring, source_nodes in rings.items():
        ring_connectivity[source_ring] = set()
        
        # Sample first 5 nodes (or all if fewer) from source ring to avoid excessive processing
        sample_nodes = source_nodes[:min(5, len(source_nodes))]
        
        for source_node in sample_nodes:
            # Use BFS to find all reachable nodes efficiently
            reachable = nx.bfs_tree(graph, source=source_node)
            
            # Check which rings are reachable
            for target_node in reachable:
                if target_node == source_node:
                    continue
                    
                target_ring = graph.nodes[target_node].get(ring_type)
                if target_ring and target_ring != source_ring:
                    ring_connectivity[source_ring].add(target_ring)
    
    return ring_connectivity

In [15]:
ring_connectivity = can_reach_between_rings(topology_logical_graph)

In [16]:
ring_connectivity

{'RING NO.04': {'RF1',
  'RF2',
  'RING NO.01',
  'RING NO.01SPUR',
  'RING NO.02',
  'RING NO.03',
  'RING NO.03SPUR',
  'RING NO.04SPUR',
  'RING NO.05',
  'RING NO.05SPUR',
  'RING NO.06',
  'RING NO.07'},
 'RING NO.01': {'RING NO.02',
  'RING NO.03',
  'RING NO.03SPUR',
  'RING NO.04',
  'RING NO.04SPUR'},
 'RING NO.02': {'RING NO.01',
  'RING NO.03',
  'RING NO.03SPUR',
  'RING NO.04',
  'RING NO.04SPUR'},
 'RING NO.03': {'RING NO.01',
  'RING NO.02',
  'RING NO.03SPUR',
  'RING NO.04',
  'RING NO.04SPUR'},
 'RING NO.03SPUR': {'RING NO.01',
  'RING NO.01SPUR',
  'RING NO.02',
  'RING NO.03',
  'RING NO.04',
  'RING NO.04SPUR',
  'RING NO.05'},
 'RING NO.04SPUR': {'RING NO.01',
  'RING NO.01SPUR',
  'RING NO.02',
  'RING NO.02SPUR',
  'RING NO.03',
  'RING NO.03SPUR',
  'RING NO.04',
  'RING NO.05',
  'RING NO.05SPUR',
  'RING NO.06',
  'RING NO.07',
  'RING NO.07 INTER BLOCK SPUR',
  'RING NO.07SPUR',
  'RING NO.08',
  'RING NO.09',
  'RING NO.09SPUR'},
 'RF1': {'RF2',
  'RING NO.